# Missing Value Imputation

The machine learning estimators in scikit-learn do not allow input or output data to contain missing values. You must handle these missing values in some fashion - by either removing them or filling them.

### Handling missing values

The quickest way to handle missing values is to simply drop the rows or columns that contain missing values. If you want to preserve all of the rows and columns in the dataset, you need to **impute** (fill) the missing values with some other value.

### Basic imputation strategies

The type of imputation you choose for your features will likely be dependent on the type of data within. The two broad types of columns are those that are continuous (always numeric) and categorical (discrete). The following describes basic imputation strategies that can quickly get your dataset prepared to do machine learning.

For continuous columns, basic strategies involve first calculating a single summary statistic for each column that contains missing values, such as the mean, median, or mode. The missing values of each column are replaced with the respective summary statistic.

For categorical columns, because there are discrete categories, often a new category is created and given a label such as 'MISSING' or some other unique value not already found in that column. Using the most frequent value is another basic strategy.

### Advanced imputation

A variety of more advanced strategies have been developed for imputation. **Hot-deck** imputation involves randomly selecting one of the non-missing values in that column for each of the missing values.

Supervised machine learning models can be used to fill in missing values by making the column with missing values the target variable and using all the other columns as the features. Currently only the k-nearest neighbors machine learning model is available for imputation.

### Iterative Imputation 

Another advanced strategy called **iterative imputation** may be used to fill in missing values in datasets where multiple columns contain missing values. As the name implies, multiple iterations take place to fill in all the missing values. A supervised machine learning model must be selected before completing the imputation. 

During the first iteration, one of the columns that contains missing values is designated as the target variable. All the other features are used as the input. Only the observations that do not contain missing values may be used. A model is trained on this subset of the data. The model then makes predictions for the missing values. It can only make predictions for observations that have no missing values.

## Imputation in scikit-learn

Only simple imputation strategies and K-nearest neighbor are available currently in scikit-learn with the `SimpleImputer` and `KNNImputer` transformer of the `impute` module. Iterative imputation is in development and should be available for scikit-learn version 0.23 expected to be released in 2020. Let's get started by reading in all of the columns of the sample housing dataset.

In [ ]:
import pandas as pd
housing = pd.read_csv('../data/housing_sample.csv')
housing.head()

### Find columns with missing values

Let's use pandas to output the number of missing values of each column.

In [ ]:
housing.isna().sum()

### Attempt to use a machine learning estimator with missing values

scikit-learn machine learning estimators do not allow for there to be a single missing value in the input or output data. (This is not entirely true anymore as of scikit-learn version 0.22. Gradient boosted trees from the ensemble module can be used with data that has missing values.) Let's select the `LotFrontage` column as one of the features in our model. 

In [ ]:
X = housing[['GrLivArea', 'GarageArea', 'LotFrontage']]
y = housing['SalePrice']

Attempting to fit a model using this data containing missing values results in an error.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, y)

### Begin with the `SimpleImputer`

One of the ways to fill in missing values is with the `SimpleImputer` transformer. It only delivers basic imputation with the mean, median, mode, or a specified constant.

### Transformer

The `SimpleImputer` is a **transformer**, a type of estimator that transforms the input data or the target variable, but does NOT do machine learning. It does **learn from data** but does not NOT learn from both the input and target simultaneously like the machine learning estimators. Typically, transformers are used to transform the input data but there are some transformers for the target variable as well.

Transformers are used **before** the machine learning models get their hands on the data. Most of the transformers are found in the `preprocessing` module as they are used to process the data before the machine learning. The `SimpleImputer` is actually found in the `impute` module which is dedicated to having tools for handling missing values.

### Import, Instantiate, Fit

All transformers follow the same three-step process as the machine learning estimators to learn from the data. In this case, the `SimpleImputer` transformer learns a single statistic from each column of the input data. During instantiation, you must set the parameter `strategy` as one of the following strings:

* 'mean'
* 'median'
* 'most_frequent'
* 'constant'

If you choose the string 'constant', you must also set the `fill_value` parameter to that constant. Below, we import, instantiate our transformer with our strategy (using the mean) and call the `fit` method to learn from the data.

In [ ]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean')
si.fit(X)

### What has it learned?

The learning is very simple. The mean was calculated from each column. Access the `statistics_` attribute to see the results of the learning.

In [ ]:
si.statistics_

You can verify this result by calling the DataFrame `mean` method.

In [ ]:
X.mean()

### No transformation has taken place yet

The input data `X` has not changed. It still contains missing values. The only thing that has taken place is the calculation of the mean of each column.

### The `transform` method

There is no `predict` method for transformers as they are not machine learning models. Instead, there exists a `transform` method that will use the information learned during the `fit` method to transform the data. We can pass the `transform` method the data we would like transformed. It returns a new copy of data that has filled the missing values. We assign this new transformed data to the variable `X_filled`.

In [ ]:
X_filled = si.transform(X)

### A numpy array is returned

Although we passed in a pandas DataFrame, scikit-learn returns us a numpy array. Let's verify this by outputting the first 5 rows.

In [ ]:
X_filled[:5]

Let's verify that no missing values remain. To do so, we must import the numpy library as numpy arrays do not have an `isna` method like DataFrames.

In [ ]:
import numpy as np
np.isnan(X_filled).sum()

The original input data is unchanged, still a pandas DataFrame, and contains the same number of missing values.

In [ ]:
X.isna().sum()

### Attempt machine learning again

We can now successfully call the `fit` method using the transformed data set.

In [ ]:
lr.fit(X_filled, y)

This new transformed data can be used in any other place in scikit-learn such as with cross validation.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
cross_val_score(lr, X_filled, y, cv=kf).mean()

### Obtain a final model by fitting on all of the data

Cross validation returns our expected score on future data, but does not give us a final model. Let's fit the linear regression model to all of this transformed data.

In [ ]:
lr.fit(X_filled, y)

### Fitting and transforming in a single step

Often, you will want to immediately transform your data after fitting it. scikit-learn provides this functionality with a single method named `fit_transform`. In this example, it finds the mean of each column and then returns a new copy of the transformed data with the missing data filled in with the mean.

In [ ]:
si = SimpleImputer(strategy='mean')
X_filled = si.fit_transform(X)

### The input data is never mutated

scikit-learn never mutates the data passed to its `transform` or `fit_transform` methods. Instead, it returns a new copy of data with the missing values filled in with the calculated strategy statistic. In our last call to `fit_transform`, the DataFrame `X` was not mutated. `X_filled` is the new copy of data with no missing values.

### Imputing new data

The goal of machine learning is to be able to make good predictions on future unseen data. It's possible this new data in the future also has missing values. In order to make predictions on it, we will have to fill those values. Let's say the following array of four observations represents new houses that we'd like to predict the sale price.

In [ ]:
X1 = np.array([[1500, 500, np.nan],
               [1200, np.nan, 88],
               [np.nan, 240, 30],
               [np.nan, np.nan, np.nan]])
X1

Our transformer `si` was already trained on the input data. It is ready to use on new data and we do not call `fit` or `fit_transform` on it again. The `transform` method will fill in the missing values with the means of the columns of the training data. Let's fill each value now.

In [ ]:
X1_filled = si.transform(X1)
X1_filled

We can verify that the correct values were filled by accessing the `statistics_` attribute again.

In [ ]:
si.statistics_

From here, we can use our linear model trained from above to make a prediction.

In [ ]:
lr.predict(X1_filled).round(-3)

### Fills in missing values for columns that had none in training

The ground living area and garage area had no missing values in the training set. Our new dataset, `X1`, had missing values for each column and even had a row of nothing but missing values. Our transformer was able to fill in these missing values since it calculated the mean on all of the columns and not just `LotFrontage`, which was the only one that had missing values in the training set.

## Common mistake - filling with mean of new data

A common mistake involves filling in the missing values of the new data with the mean of that new data. Let's calculate the mean of our new data. Because it is a numpy array, we need to use the `nanmean` function to have it ignore missing values during the calculation.

In [ ]:
np.nanmean(X1, 0)

Although it is possible for you to use these values to fill in the missing values, it is incorrect. The linear regression model was built using the means calculated on the training set and needs to be filled with those values. Also, there is a possibility that the new data has a column consisting entirely of missing values, so it would be impossible to use the mean of the new data.

## Summary of simple imputation

In summary, when we have missing data in our training set and want to fill them with a simple summary statistic, we take the following steps:

1. Instantiate the `SimpleImputer` selecting a fill strategy (in the cell below, we use the median)
1. Learn the value to be filled for each feature when calling the `fit` method
1. Using the `transform` method, create a new copy of data with the missing values filled 
1. Use cross validation to estimate future performance
1. Fit a machine learning model with this data that has no missing values
1. Get new data that you'd like to make a prediction
1. Fill in the missing values of this new data with the `transform` method (do not re-fit)
1. Call the `predict` method with the new filled data

In [ ]:
si = SimpleImputer(strategy='median')
X_filled = si.fit_transform(X)
lr = LinearRegression()
cross_val_score(lr, X_filled, y, cv=kf).mean()
lr.fit(X_filled, y)
X1_filled = si.transform(X1)
lr.predict(X1_filled).round(-3)

## K-nearest neighbor imputation

Released with scikit-learn version 0.22 in December of 2019, the `KNNImputer` provides a more advanced method of imputation. For each observation that has missing values, it finds the 'k' nearest observations using euclidean distance as its metric. It does not use the target variable in this calculation. It then predicts the missing value of that feature as the average of those 'k' nearest observations for that feature.

### Step-by-step example of KNN imputation

It's beneficial to see a step-by-step example of how KNN imputation works. Let's continue to use our same input dataset with three features, where `LotFrontage` is the only one containing missing values. First, we split the data into two separate DataFrames, `X_missing` which contains all rows with missing values, and `X_not_missing` which contains no rows with missing values.

In [ ]:
X_missing = X[X['LotFrontage'].isna()]
X_not_missing = X[X['LotFrontage'].notna()]
X_missing.head()

In [ ]:
X_not_missing.head()

We can verify that the shapes of the DataFrames are what we expect.

In [ ]:
X_missing.shape

In [ ]:
X_not_missing.shape

We need to find the nearest 'k' neighbors for each of the 259 rows in `X_missing`. Let's begin by selecting the first row of data assigning it to a new variable name.

In [ ]:
first_row = X_missing.iloc[0, :2]
first_row

We now find the distance between it's above ground living area (2,090) and the above ground living area for all of the rows in the `X_not_missing` Datarame.

In [ ]:
dist_1 = X_not_missing['GrLivArea'] - first_row['GrLivArea']
dist_1.head()

We repeat for the garage area.

In [ ]:
dist_2 = X_not_missing['GarageArea'] - first_row['GarageArea']
dist_2.head()

We calculate the euclidean distance by squaring, summing, and taking the square root of each distance.

In [ ]:
euclidean_dist = np.sqrt(dist_1 ** 2 + dist_2 ** 2)
euclidean_dist.head()

We then call the `nsmallest` method to retrieve the 'k' nearest neighbors based on this distance. Here, we choose 'k' to be 5.

In [ ]:
neighbors = euclidean_dist.nsmallest(5)
neighbors

We now ned to locate the known value of `LotFrontage` for these 5 neighbors. We can do this by using the index of this pandas Series which labels the row position.

In [ ]:
X_not_missing.loc[neighbors.index]

As expected, all of these neighbors have similar above ground living area and garage area (2,090 and 484) as the observation that had a missing `LotFrontage` value. Now, we just calculate the mean of `LotFrontage` for these five neighbors. It is this value that is used to fill in the missing value for that one observation. The process repeats for all rows that have missing values.

In [ ]:
X_not_missing.loc[neighbors.index, 'LotFrontage'].mean()

### Using `KNNImputer`

The `KNNImputer` transformer automates this process for us. Let's complete the three-step process to use KNN imputation with our data. We will immediately fit and transform the data with the `fit_transform` method.

In [ ]:
from sklearn.impute import KNNImputer
knni = KNNImputer(n_neighbors=5)
X_filled = knni.fit_transform(X)

Let's verify that the scikit-learn `KNNImputer` calculated the same result as us from above. The missing row was found at index 7.

In [ ]:
X_filled[7]

The third column of the array contains the filled value of 124.8 which was indeed the same as we calculated above.

## Iterative imputation (experimental)

Iterative imputation is actually available in scikit-learn but is labeled as 'experimental' meaning that it's not quite fully tested and that its functionality might change in the future. To enable it, run the following line.

In [ ]:
from sklearn.experimental import enable_iterative_imputer

Once you run the above, the `IterativeImputer` transformer will be available to import from the `impute` module. Let's import it now.

In [ ]:
from sklearn.impute import IterativeImputer

The `IterativeImputer` may be instantiated without setting any of its parameters. Here are explanations of some of the defaults choices it makes:

* It uses a machine learning model named Bayesian Ridge to do the imputation. You can change this by setting the first parameter, `estimator` to any supervised regression model
* It impute features in 'ascending' order - those that have the least number of missing get imputed first. Change this with the `imputation_order` parameter

Let's use iterative imputation to fill in our missing values using a decision tree with a max depth of 3 as our machine learning estimator to make the imputation. We instantiate both estimators here.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=3)
ii = IterativeImputer(dtr)

We can now learn how to impute and return a transformed dataset with the missing values filled with the `fit_transform` method.

In [ ]:
X_filled_ii = ii.fit_transform(X)
X_filled_ii[:3]

Let's look at just the rows that have missing values to see that they are no longer filled with just the same number.

In [ ]:
filt = X['LotFrontage'].isna()
X_filled_ii[filt][:5]

We can also use the trained imputer to fill in missing values from our new data.

In [ ]:
ii.transform(X1)

## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">Run the cells below which read in the full housing dataset (79) features and outputs the features with missing values and how many are missing. Do any of these columns make sense to fill in the missing values with the mean?</span>

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
housing_all = pd.read_csv('../data/housing.csv')
housing_all.head()

In [ ]:
housing_all.isna().sum()[lambda x: x > 0]

### Exercise 2

<span  style="color:green; font-size:16px">Choose a numeric column with missing values besides `LotFrontage` to be part of the model and impute missing values and then cross-validate scores with it.</span>